In [1]:
import pandas as pd
import json as json

In [2]:
def loadFiles(version, arena):
    file_path     = "../data/"
    list_of_files = []
    arena_list    = ["arena-1", "arena-2", "arena-3", "arena-4", "arena-5", "arena-6", "arena-7", "arena-8", "arena-9",]
    file_path = getVersionPath(version, file_path)
    if arena in arena_list:   
        file_path        += "/" + arena + "/"
        encounter_pokemon = file_path + "EncounterablePokemon.csv"
        pokemon_of_leader = file_path + "LeaderPokemon.csv"
        arena_leader = file_path + "ArenaLeader.csv"
        move_of_in        = file_path + "MovesOfInterest.csv"
        list_of_files.append(encounter_pokemon)
        list_of_files.append(pokemon_of_leader)
        list_of_files.append(arena_leader)
        list_of_files.append(move_of_in)  
    else:
        print("[-] Arena not found...")
    return list_of_files

def getVersionPath(version, file_path):
    if version == "red":
        file_path += "red"
        return file_path
    elif version == "blue":
        file_path += "blue"
        return file_path

In [3]:
def loadStones(arena):
    #["moon-stone", "fire-stone", "thunder-stone", "water-stone", "leaf-stone"]
    ## TODO: add arena stone data
    list_of_stones = []
    
    arena1_stones = []
    arena2_stones =  arena1_stones + ["moon-stone"]
    arena3_stones = arena2_stones + []
    
    if arena == "arena-1":
        pass
    elif arena == "arena-2":
        for stone in arena2_stones:  
            list_of_stones.append(stone)
    elif arena == "arena-3":
        for stone in arena3_stones:
            list_of_stones.append(stone)
    return list_of_stones

In [4]:
# extracts all data of interest for the enemy leader pokemon
def extractPokemonStatsFromTable(pokemonName):
    pokeRow = pokemonTable.loc[pokemonTable['Name'] == pokemonName]
    return pokeRow

In [170]:
# STATIC CSV FILES
pokemonTable = pd.read_csv('../Data/Pokemon.csv')
pokemonTypeTable = pd.read_csv('../Data/PokemonTypeChart.csv')

# SETUP ARENA-1
ARENA = 'arena-8'
GAME = 'red'
csvPaths = loadFiles(GAME, ARENA)

encounterable_pokemonTable = pd.read_csv(csvPaths[0])

# [name, moves, encounter, originIfEvo, movesByVM, movesByTM]
encounterable_pokemonParsed = []

for row in encounterable_pokemonTable.values.tolist():
    row = row[1:]
    moves = row[1]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[1] = moves
    
    VMmoves = row[4]
    VMmoves = VMmoves.replace("'", "\"") 
    VMmoves = json.loads(VMmoves)
    row[4] = VMmoves
    
    TMmoves = row[5]
    TMmoves = TMmoves.replace("'", "\"") 
    TMmoves = json.loads(TMmoves)
    row[5] = TMmoves
   
    encounterable_pokemonParsed.append(row)
    
leader_pokemon =  pd.read_csv(csvPaths[1])

## um moves von leaderPokemon zu parsen
#test = leader_pokemon.loc[leader_pokemon['name']=='geodude']['moves'].values.tolist()[0].replace("'", "\"") 
#json.loads(test)

arena_leader = pd.read_csv(csvPaths[2])
arena_leader_parsed = []
for row in arena_leader.values.tolist(): 
    moves = row[3]
    moves = moves.replace("'", "\"") 
    moves = json.loads(moves)
    row[3] = moves
    arena_leader_parsed.append(row[1:])

moves_of_interest = pd.read_csv(csvPaths[3])

## extracted Variables from DATAFRAME
AVERAGE_LEVEL = arena_leader['averageLevel'][0]

## Variables for each Arena
arena1_Options = loadStones(ARENA)

## get leaderPokemonData 
# [stats, moves]
leaderPokemonData = {}
for leaderPokemon in arena_leader_parsed[0][2]:
    moves = leader_pokemon.loc[leader_pokemon['name']==leaderPokemon]['moves'].values.tolist()[0].replace("'", "\"") 
    moves = json.loads(moves)
    moveDict = {}
    for move in moves:
        moveData = moves_of_interest.loc[moves_of_interest['name']==move]
        moveDict[move] = moveData
    leaderPokemonData[leaderPokemon] = [ extractPokemonStatsFromTable(leaderPokemon), moveDict] 


In [150]:
# spuckt liste von Pokemon aus welche zur verf. stehen

# @para1: encounterable_pokemon - liste von antreffbaren pokemon :: [index,name,encounterOrEvolutionTrigger,...]
# @para2: averageLevel - averageLevel des ArenaLeiters
# @para3: stones_available - Die Entw. Steine welche zur verfuegung stehen
# @return: poss_team_member - liste von pokemon welche mit dem averageLevel und den vorhanden Steinen erreichbar sind :: [index,name,encounterOrEvolutionTrigger,...]
def possPokemonArenaNLeaderM(encounterable_pokemon, averageLevel, stones_available):
    
    encounterable_pokemon_available = []
    for pokemon in encounterable_pokemon:
        encounterOrEvolutionTrigger = pokemon[2]
        if encounterOrEvolutionTrigger.isnumeric() and int(encounterOrEvolutionTrigger) <= averageLevel:
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger == 'walk' or encounterOrEvolutionTrigger == 'starter' or encounterOrEvolutionTrigger == 'special' or encounterOrEvolutionTrigger == 'trade':
            encounterable_pokemon_available.append(pokemon)
        elif encounterOrEvolutionTrigger in stones_available:
            encounterable_pokemon_available.append(pokemon)
    return encounterable_pokemon_available

In [151]:
def extractMoveData(moveName):
    return moves_of_interest.loc[moves_of_interest['name']==moveName]

#  extractMoveData('gust')

# returns moveTable row

In [152]:
def getMoveData(pokemonMoveNames):
    moveData = []
    for move in pokemonMoveNames:
        moveData.append(extractMoveData(move))
    return moveData

# returns array of moveTable rows

In [153]:
# extracts available move names from pokemon list item

# @param ourPokemonAndMoves:
#    [ 
#       index 0: 'name',
#       index 1: [{name:'moveName', levelLearnedAt: x}]
#       index 2: encounterOrEvoTrigger -> nur 1 starter pokemon erlaubt
#       index 3: origin if evolution -> nur 1 starter evolution erlaubt
#       index 4: movesByTM
#       index 5: movesByVM
#    ]

# @param averageLevel: int -> average LeaderPokemon level


def calcAvailableMoveNamesForPokemon(ourPokemonAndMoves, averageLevel):
    moves = []
    # LEVEL UP MOVES
    for move in ourPokemonAndMoves[1]:
        if move['levelLearnedAt'] <= averageLevel:
            moves.append(move['name'])
    # VM MOVES
    for move in ourPokemonAndMoves[4]:
        moves.append(move)
    # TM MOVES
    for move in ourPokemonAndMoves[5]:
        moves.append(move)
    return moves

# returns array with moveNames:  ['gust', 'sand-attack', 'quick-attack']

# calcAvailableMoveNamesForPokemon(possiblePokemon[0], 13)

In [154]:
def retreiveMoveEffectiveness(moveType, vsMoveType):
    return pokemonTypeTable.loc[pokemonTypeTable['type'] == moveType][vsMoveType].values[0]

# returns the effectiveness value from kaggleTable
# 0.5 | 1 | 2

In [155]:
## THIS FUNCTION IS ENTRY POINT AFTER SETTINGS

def createAvailablePokemonWithAvailableMoves(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options):
    pokeObjects = {}
    availablePokemonArrays = possPokemonArenaNLeaderM(encounterable_pokemonParsed, AVERAGE_LEVEL, arena1_Options)
    
    for pokemonArrayElement in availablePokemonArrays: 
        name = pokemonArrayElement[0]
        pokemonTableRow = extractPokemonStatsFromTable(name)
        availableMoves = calcAvailableMoveNamesForPokemon(pokemonArrayElement, AVERAGE_LEVEL)
        availableMoveDetails = getMoveData(availableMoves)
        pokeObjects[name] = {}
        pokeObjects[name]['stats'] = pokemonTableRow
        pokeObjects[name]['moves'] = availableMoveDetails
        pokeObjects[name]['origin'] = pokemonArrayElement[2]
        pokeObjects[name]['originIfEvo'] = pokemonArrayElement[3]
        pokeObjects[name]['movesVs'] = {}
        pokeObjects[name]['moveScoreVs'] = {}
    return pokeObjects

# returns { pokemonName: {stats: pokeStatTableRow, moves: availableMovesTableRow}}

In [195]:
def calcEffectiveMovesVs(ourPokeMoves, theirPoke):
    theirType1 = theirPoke[0]['Type 1'].values[0].lower()

    theirType2 = theirPoke[0]['Type 2'].values[0]
    if isinstance(theirType2, float):
        pass
    else:
        theirType2 = theirType2.lower()
    effectivenessList = []
    for move in ourPokeMoves:
        print(move)
        if len(move['moveType'].values) > 0:
            moveType = move['moveType'].values[0]
            #
            moveDamageClass = move['damageClass'].values[0]
            movePower = move['power'].values[0]
            movePP = move['pp'].values[0]
            movePrio = move['priority'].values[0]
            moveAccuracy = move['accuracy'].values[0]
            #
            
            moveName =  move['name'].values[0]
            effective = retreiveMoveEffectiveness(moveType, theirType1)
            if(moveDamageClass == 'status'):
                effective = 1.0
            effectivenessList.append({'name':moveName, 'effective':effective, 'moveType': moveType, 'moveDamageClass': moveDamageClass, 'movePower':movePower, 'movePP': movePP, 'movePrio': movePrio, 'moveAccuracy': moveAccuracy   })
        else:
            pass
    effectivenessList = sorted(effectivenessList, key=lambda d: d['effective'],reverse=True) 
    return(effectivenessList)

In [196]:
## for each arenaLeaderPokemon append the moveEffectiveness to the pokemonObjects

def appendMovesVsToPokemonObject(availablePokemonObjects, leaderPokemonData):
    for key in availablePokemonObjects:
        pokemonObject = availablePokemonObjects[key]
        for leaderPokeKey in leaderPokemonData:
            leaderPokemon = leaderPokemonData[leaderPokeKey]
            pokemonObject['movesVs'][leaderPokeKey] = calcEffectiveMovesVs(pokemonObject['moves'], leaderPokemon)

In [197]:
def appendMoveScoreToPokemonObject(ourAvailablePokemonObjects, leaderPokemonData):
     for ourName in ourAvailablePokemonObjects:
            ourPokeObject = ourAvailablePokemonObjects[ourName]
            for theirName in leaderPokemonData:
                theirPoke = leaderPokemonData[theirName]
                i = 0
                score = 0
                for move in ourPokeObject['movesVs'][theirName]:
                    if(i<4):
                        score += move['effective']
                        i += 1
                    else:
                        break
                for arenaPmove in theirPoke[1]:
                    ourPokeType1 = ourPokeObject['stats']['Type 1'].values[0].lower()
                    theirMoveType = theirPoke[1][arenaPmove]['moveType'].values[0]
                    score -= retreiveMoveEffectiveness(theirMoveType, ourPokeType1)
                ourPokeObject['moveScoreVs'][theirName] = score

In [198]:
## RUN THIS AFTER RUNNING SETTINGS 

availablePokemonObjects = createAvailablePokemonWithAvailableMoves(encounterable_pokemonParsed, 13, arena1_Options)
appendMovesVsToPokemonObject(availablePokemonObjects, leaderPokemonData)
appendMoveScoreToPokemonObject(availablePokemonObjects, leaderPokemonData)

   Unnamed: 0  name moveType damageClass  power  pp  priority  accuracy
0         0.0  gust   flying     special   40.0  35         0     100.0
   Unnamed: 0         name moveType damageClass  power  pp  priority  accuracy
1         1.0  sand-attack   ground      status    NaN  15         0     100.0
   Unnamed: 0          name moveType damageClass  power  pp  priority  \
2         2.0  quick-attack   normal    physical   40.0  30         1   

   accuracy  
2     100.0  
     Unnamed: 0 name moveType damageClass  power  pp  priority  accuracy
138         NaN  fly   flying    physical   90.0  15         0      95.0
     Unnamed: 0       name moveType damageClass  power  pp  priority  accuracy
3           3.0  whirlwind   normal      status    NaN  20        -6       NaN
142         NaN  whirlwind   normal      status    NaN  20        -6       NaN
     Unnamed: 0  name moveType damageClass  power  pp  priority  accuracy
145         NaN  bide   normal    physical    NaN  10         1   

In [200]:
for name in availablePokemonObjects:
    print(name)
    print(availablePokemonObjects[name]['moveScoreVs'])

pidgey
{'rhyhorn': 0.0, 'dugtrio': 0.0, 'nidoqueen': 0.0, 'nidoking': 0.0, 'rhydon': 0.0}
rattata
{'rhyhorn': 4.0, 'dugtrio': 4.0, 'nidoqueen': 1.0, 'nidoking': 1.0, 'rhydon': 4.0}
weedle
{'rhyhorn': -2.5, 'dugtrio': -1.5, 'nidoqueen': -2.5, 'nidoking': -2.5, 'rhydon': -2.0}
caterpie
{'rhyhorn': -2.0, 'dugtrio': -1.0, 'nidoqueen': -2.0, 'nidoking': -2.0, 'rhydon': -1.5}
metapod
{'rhyhorn': -3.0, 'dugtrio': -2.0, 'nidoqueen': -3.0, 'nidoking': -3.0, 'rhydon': -2.5}
kakuna
{'rhyhorn': -3.0, 'dugtrio': -2.0, 'nidoqueen': -3.0, 'nidoking': -3.0, 'rhydon': -2.5}
pikachu
{'rhyhorn': 1.0, 'dugtrio': -1.0, 'nidoqueen': 1.0, 'nidoking': 1.0, 'rhydon': 0.0}
spearow
{'rhyhorn': 0.0, 'dugtrio': 0.0, 'nidoqueen': 0.0, 'nidoking': 0.0, 'rhydon': 0.0}
nidoran-f
{'rhyhorn': 2.0, 'dugtrio': 0.0, 'nidoqueen': 1.5, 'nidoking': 1.5, 'rhydon': 1.0}
nidoran-m
{'rhyhorn': 2.0, 'dugtrio': 0.0, 'nidoqueen': 1.5, 'nidoking': 1.5, 'rhydon': 1.0}
jigglypuff
{'rhyhorn': 4.0, 'dugtrio': 4.0, 'nidoqueen': 4.0, 'nido